In [1]:
# Import necessary packages
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import implicit
import numpy as np
import scipy
from sklearn.neighbors import NearestNeighbors

In [2]:
# Load Data
df = pd.read_csv('lastfm_9000_users.csv', na_filter=False)
data = df.drop(['Unnamed: 0','artist_name'], axis=1)
#data.user_id = data.user_id.factorize()[0]

In [3]:
# Set up arrays of unique users and artists for our matrix
users = list(np.sort(data.user_id.unique()))
artists = list(data.artist_mbid.unique())
plays = list(data.plays)

In [4]:
#user-user
#rows2 = data.user_id.astype('category', categories=users).cat.codes
#cols2 = data.artist_mbid.astype('category', categories=artists).cat.codes
#plays_sparse2 = scipy.sparse.csr_matrix((plays, (rows2, cols2)), shape=(len(users),len(artists)))

#item-item
rows = data.artist_mbid.astype('category', categories=artists).cat.codes
cols = data.user_id.astype('category', categories=users).cat.codes
plays_sparse = scipy.sparse.csr_matrix((plays, (rows, cols)), shape=(len(artists),len(users))).astype(float)

In [5]:
sample_artists = ['kanye west',
'coldplay',
'justin timberlake']

"""
'death cab for cutie',
'paramore',
'ok go',
'foo fighters',
'green day',
'eminem',
'kanye west',
'maroon 5',
'britney spears',
'justin timberlake',
'notorious b.i.g.'

for artist in sample_artists:
    artist_index = df.artist_name[df.artist_name == artist].index.tolist()[0]
    model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
    model_knn.fit(plays_sparse)
    distances, indices = model_knn.kneighbors(plays_sparse[artist_index], n_neighbors = 6)
    print('Artists that are similar to', artist + ':')
    for i in range(0, len(indices[0])):
        print(df.artist_name[df.artist_mbid == artists[indices[0][i]]].unique())
    print('\n')
"""

"\n'death cab for cutie',\n'paramore',\n'ok go',\n'foo fighters',\n'green day',\n'eminem',\n'kanye west',\n'maroon 5',\n'britney spears',\n'justin timberlake',\n'notorious b.i.g.'\n\nfor artist in sample_artists:\n    artist_index = df.artist_name[df.artist_name == artist].index.tolist()[0]\n    model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')\n    model_knn.fit(plays_sparse)\n    distances, indices = model_knn.kneighbors(plays_sparse[artist_index], n_neighbors = 6)\n    print('Artists that are similar to', artist + ':')\n    for i in range(0, len(indices[0])):\n        print(df.artist_name[df.artist_mbid == artists[indices[0][i]]].unique())\n    print('\n')\n"

In [6]:
# Train K-Nearest Neighbors Model
model_knn = implicit.nearest_neighbours.BM25Recommender()
model_knn.fit(plays_sparse, show_progress=True)

100%|██████████| 47102/47102 [00:00<00:00, 135048.28it/s]


In [7]:
# Output Similar Artists
for artist in sample_artists:
    artist_index = df.artist_name[df.artist_name == artist].index.tolist()[0]
    related = model_knn.similar_items(artist_index)
    print('Artists that are similar to', artist + ':')
    for (x,y) in related:
        print(df.artist_name[df.artist_mbid == artists[x]].unique())
    print('\n')

Artists that are similar to kanye west:
['charles mingus']
['miles davis']
['john coltrane']
['thelonious monk']
['duke ellington']
['charlie parker']
['herbie hancock']
['sonny rollins']
['bill evans']
['tom waits']


Artists that are similar to coldplay:
['coldplay']
['radiohead']
['the killers']
['red hot chili peppers']
['muse']
['u2']
['the beatles' 'beatles']
['plan x' 'kiraç & funda arar - www.catlaemi.net'
 'bisbal y alejandro sanz' ... 'frank sinatra/duke ellington'
 'chase kreger' 'martingo']
['snow patrol']
['death cab for cutie']


Artists that are similar to justin timberlake:
['inxs']
['depeche mode']
['u2']
['the cure']
['coldplay']
['queen']
['pink floyd']
['alanis morissette']
['david bowie']
['simple minds']




In [8]:
# Generate recommendations for particular user
user = 5
for x,y in model_knn.recommend(user,plays_sparse.T.tocsr(), N=20, filter_already_liked_items=True):
    print(df.artist_name[df.artist_mbid == artists[x]].unique())

['rihanna']
['coldplay']
['linkin park']
['beyoncé' 'beyonce']
['avril lavigne']
['christina aguilera']
['michael jackson']
['madonna']
['red hot chili peppers']
['the killers']
['the beatles' 'beatles']
['amy winehouse']
['radiohead']
['muse']
['lily allen']
['timbaland']
['daft punk']
['kelly clarkson']
['lady gaga']
['kanye west']
